In [141]:
from scrapfly import ScrapflyClient, ScrapeConfig, ScrapflyError
 
scrapfly = ScrapflyClient(key="scp-live-01fc8534987042f4a2fe553c5cf6a2df", max_concurrency=2)
url = 'https://mikescamera.com/shop?query=91354'
listing_result = scrapfly.scrape(ScrapeConfig(url=url, render_js=True, country="US", asp=True, retry=False, rendering_wait=10000))
listing_html = listing_result.scrape_result['content']

In [142]:
from bs4 import BeautifulSoup
import re
import json
def extract_json_from_html(html_content):
    soup = BeautifulSoup(html_content, 'html.parser')
    script_tags = soup.find_all('script')

    json_data_list = []

    for script in script_tags:
        script_content = script.string
        if script_content:
            # Use a regular expression to find JSON-like content
            json_pattern = re.compile(r'({.*})', re.DOTALL)
            matches = json_pattern.findall(script_content)
            for match in matches:
                try:
                    json_data = json.loads(match)
                    json_data_list.append(json_data)
                except json.JSONDecodeError:
                    pass  # Ignore invalid JSON

    return json_data_list

json_data_list = extract_json_from_html(listing_html)

In [143]:
sellername = None
site_url = None

for index in range(2):  # Assuming only the first two indices are relevant
    # Ensure the index is valid and that the 'name' is not None
    if index < len(json_data_list) and json_data_list[index].get('name') is not None:
        # Check if the current product name matches the desired one
            data = json_data_list[index]
            
            # Attempt to populate sellername and site_url if they exist
            if index + 1 < len(json_data_list):
                sellername = json_data_list[index + 1].get('@graph', [{}])[0].get('name')
                site_url = json_data_list[index + 1].get('@graph', [{}])[0].get('url')

            break


In [144]:
result = {
    'ProductName': data.pop('name'),
    'description': data.pop('description'),
    'sku': data.pop('sku'),
    'brand': data['brand']['name'],  # Rename to 'Brand name'
    'color': data.get('color'),  # Optional field, adjust accordingly
    'image': data.pop('image')[0],  # Assuming the first image is needed
    'priceCurrency': data['offers']['priceCurrency'],
    'price': data['offers']['price'],
    'availability': data['offers']['availability'],
    'url': data['offers'].get('url'),
    'mpn': data.pop('mpn'),
    'sellername' : sellername,
    'siteurl' : site_url
}

# Print the resulting dictionary
print(json.dumps(result, indent=2))

{
  "ProductName": "EOS R50 Mirrorless Camera - Content Creator Kit",
  "description": "<strong>Capture the Moment Just as You Imagine It</strong><br>The EOS R50 camera makes photography fun, simple, and convenient with intuitive photo functions that capture beautiful images automatically, just as you imagine them. A host of video functions such as 6K oversampled 4K 30p capture and Dual Pixel CMOS AF II makes the EOS R50 a great camera for movie recording and vlogging as well. The 24.2 Megapixel APS-C size CMOS Image sensor and DIGIC X image processor ensure high image quality in a compact and lightweight form factor.\u00a0<br><br><strong>FEATURES<br>24.2 Megapixel (APS-C) CMOS Sensor</strong><br>Equipped with a 24.2-million-pixel CMOS (APS-C) image sensor and DIGIC X Image Processor, the EOS R50 camera can capture images and videos with reduced noise, delivering clear, detailed results even in low light. Images look beautiful even when enlarged, with natural bokeh that smartphones can